# Betrayal Detection in Diplomacy

Diplomacy is a strategic board game with social deception elements. The game consists of phases where players may negotiate and betray each other. We will attempt to predict whether betrayal occurs based off of the messages that players send to each other.

In [265]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import classification_report,confusion_matrix

print(df.info()) # format

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   seasons   500 non-null    object
 1   game      500 non-null    int64 
 2   betrayal  500 non-null    bool  
 3   idx       500 non-null    int64 
 4   people    500 non-null    object
dtypes: bool(1), int64(2), object(2)
memory usage: 16.2+ KB
None


# Basic Preprocessing

First we preprocess the dataset into a dataframe that is easier to interpret for exploration/training.

In Diplomacy, each game is divided into multiple phases called seasons. Within each season, players can communicate with each other to coordinate attacks. Every entry in the original dataset consists of a particular interaction between two players in a game, containing statistical data regarding their messages spread across all seasons.

Therefore, in our preprocessed dataframe we will aggregate all message data per each season of the game, as we are only concerned about whether or not there was a betrayal in the interaction for the game as a whole.

In [266]:
df = pd.read_json('diplomacy_data.json')
# entries represent an interaction between two players in a particular game, spread across
# all of the games seasons

data = [] # 2d array to hold processed data

# we create a new processed pandas dataframe based off of the old one
'''
Our processed data will contain the following structure:

ID | Betrayal (T/F) | victim # msgs | betrayer # msgs |
victim # sentences | betrayer # sentences |
victim # words | betrayer # words | victim avg words per msg | betrayer avg words per msg | 
victim # requests | betrayer # requests | victim avg politeness | betrayer avg politeness | 
victim neg sentiment proportion | betrayer neg sentiment proportion |
victim neu sentiment proportion | betrayer neu sentiment proportion | 
victim pos sentiment proportion | betrayer pos sentiment proportion | 

Note that if no betrayal occurs, the 'betrayer' label indicates the other person in the interaction,
chosen arbitrarily

'''

betrayals = df['betrayal']
game_session_seasons = df['seasons']

# create our new processed df

for i, seasons in enumerate(game_session_seasons):

    # print("new entry")
    
    entry = [] # represents a row of our new pandas dataframe

    # if this interaction is a betrayal
    betrayal = betrayals[i]
    entry.append(betrayal)

    # initialize all data that needs to be kept track of 
    v_msgs, b_msgs = 0, 0
    v_sentences, b_sentences = 0, 0
    v_words, b_words = 0, 0
    v_words_per_msg, b_words_per_msg = 0, 0
    v_requests, b_requests = 0, 0
    v_avg_polite, b_avg_polite = 0, 0
    v_neg_perc, b_neg_perc = 0, 0
    v_net_perc, b_net_perc = 0, 0
    v_pos_perc, b_pos_perc = 0, 0

    for season in seasons:
        # print(season)

        v_msg_list = season['messages']['victim']
        b_msg_list = season['messages']['betrayer']

        if len(v_msg_list) > 0:
            v_msgs += len(v_msg_list)

            # track victim message stats
            for msg in v_msg_list:
                v_sentences += msg['n_sentences']
                v_words += msg['n_words']
                v_requests += msg['n_requests']
                v_avg_polite += msg['politeness']
                v_neg_perc += msg['sentiment']['negative']
                v_net_perc += msg['sentiment']['neutral']
                v_pos_perc += msg['sentiment']['positive']
                
                
        if len(b_msg_list) > 0:
            b_msgs += len(b_msg_list)

            # track betrayer message stats
            for msg in b_msg_list:
                b_sentences += msg['n_sentences']
                b_words += msg['n_words']
                b_requests += msg['n_requests']
                b_avg_polite += msg['politeness']
                b_neg_perc += msg['sentiment']['negative']
                b_net_perc += msg['sentiment']['neutral']
                b_pos_perc += msg['sentiment']['positive']
                
    entry.append(v_msgs) # total amount of victim msgs for all seasons in this entry
    entry.append(b_msgs)
    
    entry.append(v_sentences)
    entry.append(b_sentences)
    
    entry.append(v_words)
    entry.append(b_words)
    
    if v_msgs != 0:
        v_words_per_msg = v_words/v_msgs
    
    if b_msgs != 0:
        b_words_per_msg = b_words/b_msgs
    
    # remove entries with no conversation
    if v_msgs == 0 and b_msgs == 0:
        continue
    
    entry.append(v_words_per_msg)
    entry.append(b_words_per_msg)
    
    entry.append(v_requests)
    entry.append(b_requests)
    
    if v_msgs != 0:
        v_avg_polite = v_avg_polite/v_msgs
        
    if b_msgs != 0:
        b_avg_polite = b_avg_polite/b_msgs
        
    entry.append(v_avg_polite)
    entry.append(b_avg_polite)
        
    if v_sentences != 0:
        v_neg_perc = v_neg_perc/v_sentences
        v_net_perc = v_net_perc/v_sentences
        v_pos_perc = v_pos_perc/v_sentences
        
    entry.append(v_neg_perc)
    entry.append(v_net_perc)
    entry.append(v_pos_perc)
    
    if b_sentences != 0:
        b_neg_perc = b_neg_perc/b_sentences
        b_net_perc = b_net_perc/b_sentences
        b_pos_perc = b_pos_perc/b_sentences
        
    entry.append(b_neg_perc)
    entry.append(b_net_perc)
    entry.append(b_pos_perc)
    
    data.append(entry)

print(len(data))

# create a new dataframe of our processed data
columns = ['betrayal', 'victim_msgs', 'betrayer_msgs', 'victim_sentences', 'betrayer_sentences', 
           'victim_words', 'betrayer_words', 'victim_words_per_msg', 'betrayer_words_per_msg',
          'victim_reqs', 'betrayer_reqs', 'victim_avg_polite', 'betrayer_avg_polite',
          'victim_neg_perc', 'betrayer_neg_perc', 'victim_neu_perc', 'betrayer_neu_perc',
          'victim_pos_perc', 'betrayer_pos_perc']

new_df = pd.DataFrame(data, columns=columns)
print(len(new_df))
# print(new_df)
new_df.to_csv('447processed.csv')

410
410


PermissionError: [Errno 13] Permission denied: '447processed.csv'